In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('Housing_Prices/train.csv')

data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Create X and y
y = data[['SalePrice']]
X = data.drop(columns=['SalePrice', 'Id'], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [4]:
# Save y
y_train.to_csv('data/y_train.csv', index=False)
y_val.to_csv('data/y_val.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

In [5]:
# Remove "object"-type features from X
cont_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]

# Remove "object"-type features from X_train and X_test
X_train_cont = X_train.loc[:, cont_features]
X_val_cont = X_val.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median')
X_train_numeric = pd.DataFrame(impute.fit_transform(X_train_cont), columns=cont_features)
X_val_numeric = pd.DataFrame(impute.transform(X_val_cont), columns=cont_features)
X_test_numeric = pd.DataFrame(impute.transform(X_test_cont), columns=cont_features)

In [6]:
# Save numeric X
X_train_numeric.to_csv('data/X_train_numeric.csv', index=False)
X_val_numeric.to_csv('data/X_val_numeric.csv', index=False)
X_test_numeric.to_csv('data/X_test_numeric.csv', index=False)

In [7]:
# Create X_cat which contains only the categorical variables
features_cat = [col for col in X.columns if X[col].dtype in [np.object]]
X_train_cat = X_train.loc[:, features_cat]
X_val_cat = X_val.loc[:, features_cat]
X_test_cat = X_test.loc[:, features_cat]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_val_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

In [8]:
# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_val_ohe = ohe.transform(X_val_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame 
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)
X_val_cat = pd.DataFrame(X_val_ohe.todense(), columns=cat_columns)
X_test_cat = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

In [9]:
# Save categorical X
X_train_cat.to_csv('data/X_train_cat.csv', index=False)
X_val_cat.to_csv('data/X_val_cat.csv', index=False)
X_test_cat.to_csv('data/X_test_cat.csv', index=False)